In [51]:
#Essential to have this equal to scaling during training.
RNN_INPUTS_SCALING_FACTOR = 1
SIGMA_TEMP = 0.01

In [52]:
import keras
import tensorflow as tf

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

#Importing the VAE and RNN.
import os
import sys
#Adding WorldModels path to pythonpath
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

/home/kaiolae/code/word_models_keras_test/WorldModels


In [53]:
#To access gym
import sys
sys.path.append('/home/kaiolae/.virtualenvs/worldmodels/lib/python3.5/site-packages')
import os

In [54]:

import analyze_rnn_predictions
%load_ext autoreload
%autoreload 2 #If we want to automatically load external modules as they are updated.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML


def plot_movie_mp4(image_array):
    dpi = 2.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    #fig = plt.figure(figsize=(1,1), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    display(HTML(anim.to_html5_video()))

In [56]:
loading_model_path = "../trained_sequential_rnn_4mixtures_run2/rnn_trained_model.h5"
rnn_analyzer = analyze_rnn_predictions.RNNAnalyzer(loading_model_path, 4, 1.0, io_scaling = RNN_INPUTS_SCALING_FACTOR)

VAE init
Shape before flattening: (None, 16, 16, 32)
KL Shape: (None,)
Xent shape: ()
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 64, 64, 3)    39          encoder_input[0][0]              
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 32, 32, 32)   416         conv2d_17[0][0]                  
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 16, 16, 32)   4128        conv2d_18[0][0]                  
_______________________

In [57]:
#Get a start video
#Getting data to feed into the VAE and RNN
import numpy as np
import os
# load preprocessed data
training_data = np.load(os.path.join("../rnn_data_64_dim", "rnn_training_data.npz"))
action_data = training_data['action']
observation_data = training_data['latent']

episode_num = 52
single_action_sequence = action_data[episode_num]
single_latent_sequence = observation_data[episode_num]
print("Actions length: ", len(single_action_sequence))
print("Latent vectors length: ", len(single_latent_sequence))
num_frames = len(single_latent_sequence)


#Decode all 100 vectors, store as video
#Visualize video.

Actions length:  300
Latent vectors length:  300


In [58]:
#Decode a sequence with the VAE and visualize it
def decode_and_visualize(latent_vector_sequence, rnn_analyzer):
    plot_movie_mp4(rnn_analyzer.decode_with_vae(latent_vector_sequence))

### Making a dream with the trained RNN

In [59]:
import random
def generate_random_action():
    return random.uniform(-1.0,1.0)


In [60]:
rnn_analyzer._reset()
print(len(single_latent_sequence))
rnn_analyzer.warm_up_lstm(single_action_sequence[:150], single_latent_sequence[:150])
dreamed_latents = []
#TODO Rather than action 0, select random action.
dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), previous_z=single_latent_sequence[150], sigma_temp=SIGMA_TEMP)
dreamed_latents.append(dreamed_latent)

num_timesteps=100
for i in range(num_timesteps-1):
    dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), sigma_temp=SIGMA_TEMP)
    dreamed_latents.append(dreamed_latent)
decode_and_visualize(dreamed_latents, rnn_analyzer)

300


<Figure size 64x64 with 0 Axes>

## Committing to one mixture during dreaming: 0,1,2,3

In [61]:
#Committed dreaming
rnn_analyzer._reset()
print(len(single_latent_sequence))
rnn_analyzer.warm_up_lstm(single_action_sequence[:150], single_latent_sequence[:150])
dreamed_latents = []
#TODO Rather than action 0, select random action.
dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), previous_z=single_latent_sequence[150], sigma_temp=SIGMA_TEMP,force_prediction_from_mixture = 0)
dreamed_latents.append(dreamed_latent)

num_timesteps=100
for i in range(num_timesteps-1):
    dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), sigma_temp=SIGMA_TEMP, force_prediction_from_mixture = 0)
    dreamed_latents.append(dreamed_latent)
decode_and_visualize(dreamed_latents, rnn_analyzer)

300


<Figure size 64x64 with 0 Axes>

In [62]:
#Committed dreaming
rnn_analyzer._reset()
print(len(single_latent_sequence))
rnn_analyzer.warm_up_lstm(single_action_sequence[:150], single_latent_sequence[:150])
dreamed_latents = []
#TODO Rather than action 0, select random action.
dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), previous_z=single_latent_sequence[150], sigma_temp=SIGMA_TEMP,force_prediction_from_mixture = 1)
dreamed_latents.append(dreamed_latent)

num_timesteps=100
for i in range(num_timesteps-1):
    dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), sigma_temp=SIGMA_TEMP, force_prediction_from_mixture = 1)
    dreamed_latents.append(dreamed_latent)
decode_and_visualize(dreamed_latents, rnn_analyzer)

300


<Figure size 64x64 with 0 Axes>

In [63]:
#Committed dreaming
rnn_analyzer._reset()
print(len(single_latent_sequence))
rnn_analyzer.warm_up_lstm(single_action_sequence[:150], single_latent_sequence[:150])
dreamed_latents = []
#TODO Rather than action 0, select random action.
dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), previous_z=single_latent_sequence[150], sigma_temp=SIGMA_TEMP,force_prediction_from_mixture = 2)
dreamed_latents.append(dreamed_latent)

num_timesteps=100
for i in range(num_timesteps-1):
    dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), sigma_temp=SIGMA_TEMP, force_prediction_from_mixture = 2)
    dreamed_latents.append(dreamed_latent)
decode_and_visualize(dreamed_latents, rnn_analyzer)

300


<Figure size 64x64 with 0 Axes>

In [64]:
#Committed dreaming
rnn_analyzer._reset()
print(len(single_latent_sequence))
rnn_analyzer.warm_up_lstm(single_action_sequence[:150], single_latent_sequence[:150])
dreamed_latents = []
#TODO Rather than action 0, select random action.
dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), previous_z=single_latent_sequence[150], sigma_temp=SIGMA_TEMP,force_prediction_from_mixture = 3)
dreamed_latents.append(dreamed_latent)

num_timesteps=100
for i in range(num_timesteps-1):
    dreamed_latent, mixture_weights = rnn_analyzer.predict_one_step(generate_random_action(), sigma_temp=SIGMA_TEMP, force_prediction_from_mixture = 3)
    dreamed_latents.append(dreamed_latent)
decode_and_visualize(dreamed_latents, rnn_analyzer)

300


<Figure size 64x64 with 0 Axes>